In [1]:
import torch 
import numpy as np
import pandas as pd
import tensorboard
from torch.nn.utils.rnn import pad_sequence


In [2]:
import sys
sys.path.append('C:/Users/Owner/ICT5012 - Disseration/transit_learning-master_Newest')
from torch_geometric.loader import DataLoader
from simulation.citygraph_dataset import CityGraphData, \
    get_dataset_from_config, STOP_KEY
from simulation.transit_time_estimator import RouteGenBatchState, get_cost_module_from_cfg
import learning.utils as lrnu
from torch_utils_2 import get_batch_tensor_from_routes, dump_routes
from tqdm import tqdm
import time

In [3]:
# ANDRE - CHANGE HERE
SOUTHERN_HARBOUR_ROUTES = [
    torch.tensor([188,189,190,191,192,182]),
    torch.tensor([164,39,27,28,30,31,32,33,34,35,36]),
    torch.tensor([166,39,40,28,30,31,32,33,34,35,36]),
    torch.tensor([68,56,57,58,176,177,178,179,180,181,182]),
    torch.tensor([139,120]),
    torch.tensor([182,181,180,179,193,176,194,67,68]),
    torch.tensor([36,73]),
    torch.tensor([139,120]),
    torch.tensor([92,93,94]),
    torch.tensor([139,120]),
    torch.tensor([139,120]),
    torch.tensor([95,96,97,98,99,100,101,102,103,104,105,106,107,108,109]),
    torch.tensor([139,120]),
    torch.tensor([120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,24,25,205,203,202,201,140,144,200,142,141,146,147,199,145,195,196,143,2,5,197,149,206,43,44,45,7,46,85,86,87,88,89,76,83,90,82,91,111,94]),
    torch.tensor([38,223,224,225,226,227,228,229,230,236,237,238,239,240,241]),
    torch.tensor([109,110,111,118,117,116,119,214,213,215,56,57,58,59,60,61,62,169,170,171,172,173,174,16,17,13,12,18,11,10,9,19,8,64]),
    torch.tensor([6,7,152,153,150,151,23,0,1,2,3,143,196,195,198,145,199,147,146,148,141,142,200,144,140,201,202,203,204,24,37,4,138,139]),
    torch.tensor([139,120]),
    torch.tensor([68,56,213,214,119,115,114,113,76,83,84,75,74,50,49,48,47,70,7,22,21,165,154,155,156,157,158,159,160,161,162,163]),
    torch.tensor([182,183,184,185,186,187,188]),
    torch.tensor([139,120]),
    torch.tensor([163,26,20,165,21,22,69,70,7,71,43,42,41,29,30,31,72,33,34,35,36]),
    torch.tensor([175,167,209,210,211,212,168,14,15,65,63,62,66,60,59,67,68]),
    torch.tensor([112,39,216,217,218,219,220,221,222,208,207,36]),
    torch.tensor([68,56,55,54,53,52,51,50,74,75,76,77,78,79,80,81])
]


Northern_Harbour_Routes = [
torch.tensor([278,279,280,281,282,272,271,270,269,283,266,284,97,98,94,93,92,91,90,89,88,87,86,85,99,15,264,265,241,242]),
torch.tensor([17,47,5,48,6,49,7,50,8,78,9,51,14,52,53,54,13,55,56,57,58,59,295,296,297,222,224,223]),
torch.tensor([17,47,5,48,6,49,7,50,8,78,9,51,14,52,53,54,13,55,56,57,58,59,295,296,297,222,224,223]),
torch.tensor([17,47,5,48,6,49,7,50,8,78,9,51,14,52,53,54,13,55,56,57,58,59,295,296,297,222,224,223]),
torch.tensor([17,47,5,48,6,49,7,50,8,78,9,51,14,52,53,54,13,55,56,57,58,59,295,296,297,222,224,223]),
torch.tensor([242,16,195,196,68,69,33,70,73,74,75,67,4,186,187,188,189,172,190,175,174,173]),
torch.tensor([17,47,5,48,6,49,7,50,8,78,9,51,14,52,53,54,13,55,56,57,58,59,295,296,297,222,224,223]),
torch.tensor([185,101,64,65,66,67,4,186,187,188,189,172,190,175,174,173]),
torch.tensor([278,279,280,281,282,272,271,270,269,283,266,284,97,98,94,93,92,91,90,89,88,87,86,85,99,15,264,265,241,242]),
torch.tensor([17,47,5,48,6,49,7,50,8,78,9,51,14,52,53,54,13,55,56,57,58,59,295,296,297,222,224,223]),
torch.tensor([157,158,153,152,151,150,34,35,11,68,129,74,75,67,197,0,1,2,3,177,292,237]),
torch.tensor([17,47,5,48,6,49,7,50,8,78,9,51,14,52,53,54,13,55,56,57,58,59,295,296,297,222,224,223]),
torch.tensor([17,47,5,48,6,49,7,50,8,78,9,51,14,52,53,54,13,55,56,57,58,59,295,296,297,222,224,223]),
torch.tensor([17,47,5,48,6,49,7,50,8,78,9,51,14,52,53,54,13,55,56,57,58,59,295,296,297,222,224,223]),
torch.tensor([17,47,5,48,6,49,7,50,8,78,9,51,14,52,53,54,13,55,56,57,58,59,295,296,297,222,224,223]),
torch.tensor([229,63,41,205,206,207,208,209,210,211,212,213,214,215,216,219,259]),
torch.tensor([242,16,195,196,68,69,70,32,73,74,75,67,4,186,187,188,189,172,190,175,174,173]),
torch.tensor([17,47,5,48,6,49,7,50,8,78,9,51,14,52,53,54,13,55,56,57,58,59,295,296,297,222,224,223]),
torch.tensor([17,47,5,48,6,49,7,50,8,78,9,51,14,52,53,54,13,55,56,57,58,59,295,296,297,222,224,223]),
torch.tensor([128,101,64,65,66,67,4,186,187,188,189,171,170,191,192,193,293,17]),
torch.tensor([17,47,5,48,6,49,7,50,8,78,9,51,14,52,53,54,13,55,56,57,58,59,295,296,297,222,224,223]),
torch.tensor([40,63,41,205,206,207,208,209,210,211,212,213,214,215,216,219,259]),
torch.tensor([17,47,5,48,6,49,7,50,8,78,9,51,14,52,53,54,13,55,56,57,58,59,295,296,297,222,224,223]),
torch.tensor([17,47,5,48,6,49,7,50,8,78,9,51,14,52,53,54,13,55,56,57,58,59,295,296,297,222,224,223]),
torch.tensor([102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,17]),
torch.tensor([17,47,5,48,6,49,7,50,8,78,9,51,14,52,53,54,13,55,56,57,58,59,295,296,297,222,224,223]),
torch.tensor([17,47,5,48,6,49,7,50,8,78,9,51,14,52,53,54,13,55,56,57,58,59,295,296,297,222,224,223]),
torch.tensor([17,47,5,48,6,49,7,50,8,78,9,51,14,52,53,54,13,55,56,57,58,59,295,296,297,222,224,223]),
torch.tensor([17,47,5,48,6,49,7,50,8,78,9,51,14,52,53,54,13,55,56,57,58,59,295,296,297,222,224,223]),
torch.tensor([17,47,5,48,6,49,7,50,8,78,9,51,14,52,53,54,13,55,56,57,58,59,295,296,297,222,224,223]),
torch.tensor([17,47,5,48,6,49,7,50,8,78,9,51,14,52,53,54,13,55,56,57,58,59,295,296,297,222,224,223]),
torch.tensor([17,47,5,48,6,49,7,50,8,78,9,51,14,52,53,54,13,55,56,57,58,59,295,296,297,222,224,223]),
torch.tensor([244,63,41,205,206,207,208,209,210,211,212,213,214,215,216,219,259]),
torch.tensor([17,47,5,48,6,49,7,50,8,78,9,51,14,52,53,54,13,55,56,57,58,59,295,296,297,222,224,223]),
torch.tensor([244,63,41,205,206,207,208,209,210,211,212,213,214,215,216,217,218,204,289,202,203,179,180,181,182]),
torch.tensor([102,103,123,124,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,17])
]


SOUTH_EASTERN_ROUTES = [
    torch.tensor([117,79,82,61,72,62,83,84,85,118,119,120,121,122,123,124,125,126,127,1,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144]),
    torch.tensor([144,143,142,145,140,139,137,136,146,147,134,133,132,148,149,130,129,128,1,127,125,121,120,119,118,85,83,62,72,73,114,75,76,77,78]),
    torch.tensor([150,79,82,61,72,62,83,84,85,118,119,120,121,122,123,124,125,126,127,1,128,129,130,151,152,153,154,155,156,157,158,159,160,161,162,163,164]),
    torch.tensor([163,162,161,160,159,158,157,156,155,154,153,152,151,130,129,128,1,127,125,121,120,119,118,85,83,62,72,73,114,75,76,77,78]),
    torch.tensor([117,79,82,61,72,62,83,84,85,118,165,3,63,64,65,66,5,128,129,166,167,168,134,135,136,137,138,139,140,169,170,171,172,173,174,175,176,177,178,144]),
    torch.tensor([144,143,142,145,140,139,137,136,146,147,134,168,167,166,129,128,1,2,66,65,63,3,165,118,85,83,62,72,73,114,75,76,77,78]),
    torch.tensor([169,170,171,172,173,174,175,176,177,178,144,143,142,145,140,139,137,136,146,147,134,168,167,166,129,128,1,2,66,65,63,3,165,118,85,83,62,72,73,114,75,76,77,78]),
    torch.tensor([150,79,82,61,72,62,115,3,63,179,180,181,182,183,184,185,158,157,156,186,160,161,162,164,163,187,188,189,190,191,192,193,144]),
    torch.tensor([144,193,192,191,190,189,188,194,195,187,164,163,162,161,160,186,156,157,158,185,184,183,182,181,180,196,3,116,62,72,73,114,75,76,77,78]),
    torch.tensor([197,81,82,61,72,62,83,84,85,87,198,199,200,201,202,203,204,205,206,207,58,56,208,209,210,211,212,213,214,215,216,50,217,218,219,19,18,51,52,53,54,16]),
    torch.tensor([15,16,17,18,19,219,218,217,30,31,214,213,211,212,210,209,220,221,208,198,199,200,201,202,203,204,205,206,207,58,56,87,85,83,62,72,73,114,75,76,77,78]),
    torch.tensor([222,81,82,61,72,62,83,84,85,86,87,7,57,223,224,225,226,209,227,216,50,217,218,219,19,18,51,52,53,54,16,55,14,228,229,230,231,232,233,234,235,14]),
    torch.tensor([15,16,17,18,19,219,218,217,30,236,227,209,220,221,226,225,237,238,57,111,112,7,113,85,83,62,72,73,114,75,76,77,78]),
    torch.tensor([232,233,234,235,14,15,16,17,18,19,219,218,217,30,236,227,209,220,221,226,225,237,238,57,111,112,7,113,85,83,62,72,73,114,75,76,77,78]),
    torch.tensor([52,53,54,230,231,232,233,234,235,14,15,16,17,18,19,219,218,217,30,236,227,209,220,221,226,225,237,238,57,111,112,7,113,85,83,62,72,73,114,75,76,77,78]),
    torch.tensor([197,81,82,61,72,62,83,84,85,86,87,7,57,223,224,225,240,241,242,243,244,245,246,247,248,249,211,250,251,252,253,254,255,34,48,49,33,32,214,213,211,212]),
    torch.tensor([212,249,248,247,246,245,256,243,242,241,240,237,238,57,111,112,7,113,85,83,62,72,73,74,75,76,77,78]),
    torch.tensor([211,250,251,252,253,254,255,34,48,49,33,32,214,213,212,249,248,247,246,245,256,243,242,241,240,237,238,57,111,112,7,113,85,83,62,72,73,74,75,76,77,78]),
    torch.tensor([257,81,82,61,72,62,83,84,85,87,258,259,223,224,225,226,209,210,211,212,213,214,215,216,50,217,218,219,20,21,22,23,24,25,26,27,28,29]),
    torch.tensor([239,81,82,61,72,62,83,84,85,86,87,7,57,223,224,225,226,209,227,216,260,261,262,263,264,265,266,267,268,269,270]),
    torch.tensor([271,272,273,267,266,265,264,263,262,261,274,236,227,209,220,221,226,225,237,238,57,111,112,7,113,85,83,62,72,73,114,75,76,77,78]),
    torch.tensor([80,81,82,61,72,62,83,84,85,86,87,7,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,43,44,45,41,40,46,38,37,36,47,35,106]),
    torch.tensor([42,43,107,105,104,108,102,101,109,110,98,97,96,95,94,93,92,91,90,89,111,112,7,113,85,83,62,72,73,114,75,76,77,78]),
    torch.tensor([44,45,41,40,46,38,37,36,47,35,106,42,43,107,104,108,102,101,109,110,98,97,96,95,94,93,92,91,90,89,111,112,7,113,85,83,62,72,73,114,75,76,77,78]),
    torch.tensor([275,81,82,61,72,62,83,84,85,86,87,7,88,89,90,91,92,93,94,95,96,276,277,278,279,280,281,282,283,284,42,43,107,105,285,286,287,288,289,290,44]),
    torch.tensor([291,283,292,281,293,279,278,276,96,95,94,93,92,91,90,89,111,112,7,113,85,83,62,72,73,114,75,76,77,78]),
    torch.tensor([286,287,288,289,290,44,291,283,292,281,293,279,278,276,96,95,94,93,92,91,90,89,111,112,7,113,85,83,62,72,73,114,75,76,77,78]),
    torch.tensor([275,81,82,61,72,62,83,84,85,86,87,7,88,89,90,91,92,93,94,95,96,276,277,278,279,280,281,282,283,284,42,43,44,294,295,296,297,38,39,40,41]),
    torch.tensor([291,283,292,281,293,279,278,276,96,95,94,93,92,91,90,89,111,112,7,113,85,83,62,72,73,114,75,76,77,78]),
    torch.tensor([294,295,296,297,38,39,40,41,291,283,292,281,293,279,278,276,96,95,94,93,92,91,90,89,111,112,7,113,85,83,62,72,73,114,75,76,77,78]),
    torch.tensor([59,60,61,62,4,3,63,64,65,66,5,2,6,8,9,67,68,69,70,71,10,11,12,13,14,15,16,17,18,51,52,53,54,16]),
    torch.tensor([55,14,13,12,10,71,70,69,68,67,9,0,1,2,66,65,63,3,4,62,72,73,74,75,76,77,78]),
    torch.tensor([71,70,69,68,67,9,0,1,2,66,65,63,3,4,62,72,73,74,75,76,77,78])
]

WESTERN_ROUTES = [
torch.tensor([288,289,290,294,295,296,297,298,282,299,316,254,257,258,259,317,18,19,20,219,220,221,222,323,208,209]),
torch.tensor([288,289,290,294,295,296,297,298,282,299,316,254,257,258,259,317,18,19,20,219,220,221,222,323,208,209]),
torch.tensor([288,289,290,294,295,296,297,298,282,299,316,254,257,258,259,317,18,19,20,219,220,221,222,323,208,209]),
torch.tensor([288,289,290,294,295,296,297,298,282,299,316,254,257,258,259,317,18,19,20,219,220,221,222,323,208,209]),
torch.tensor([183,174,173,172,171,170,169,168,149,148,147,130,129,128,38,19,127,39,139,126,259,260,261,262,263,264,265,266]),
torch.tensor([288,289,290,294,295,296,297,298,282,299,316,254,257,258,259,317,18,19,20,219,220,221,222,323,208,209]),
torch.tensor([288,289,290,294,295,296,297,298,282,299,316,254,257,258,259,317,18,19,20,219,220,221,222,323,208,209]),
torch.tensor([288,289,290,294,295,296,297,298,282,299,316,254,257,258,259,317,18,19,20,219,220,221,222,323,208,209]),
torch.tensor([288,289,290,294,295,296,297,298,282,299,316,254,257,258,259,317,18,19,20,219,220,221,222,323,208,209]),
torch.tensor([288,289,290,294,295,296,297,298,282,299,316,254,257,258,259,317,18,19,20,219,220,221,222,323,208,209]),
torch.tensor([288,289,290,294,295,296,297,298,282,299,316,254,257,258,259,317,18,19,20,219,220,221,222,323,208,209]),
torch.tensor([288,289,290,294,295,296,297,298,282,299,316,254,257,258,259,317,18,19,20,219,220,221,222,323,208,209]),
torch.tensor([288,289,290,294,295,296,297,298,282,299,316,254,257,258,259,317,18,19,20,219,220,221,222,323,208,209]),
torch.tensor([288,289,290,294,295,296,297,298,282,299,316,254,257,258,259,317,18,19,20,219,220,221,222,323,208,209]),
torch.tensor([288,289,290,294,295,296,297,298,282,299,316,254,257,258,259,317,18,19,20,219,220,221,222,323,208,209]),
torch.tensor([288,289,290,294,295,296,297,298,282,299,316,254,257,258,259,317,18,19,20,219,220,221,222,323,208,209]),
torch.tensor([288,289,290,294,295,296,297,298,282,299,316,254,257,258,259,317,18,19,20,219,220,221,222,323,208,209]),
torch.tensor([288,289,290,291,292,293,294,295,296,297,298,282,299,316,254,257,258,259,317,18,19,20,219,220,221,222,323,208,209]),
torch.tensor([288,289,290,294,295,296,297,298,282,299,316,254,257,258,259,317,18,19,20,219,220,221,222,323,208,209]),
torch.tensor([288,289,290,294,295,296,297,298,282,299,316,254,257,258,259,317,18,19,20,219,220,221,222,323,208,209]),
torch.tensor([288,289,290,294,295,296,297,298,282,299,316,254,257,258,259,317,18,19,20,219,220,221,222,323,208,209]),
torch.tensor([288,289,290,294,295,296,297,298,282,299,316,254,257,258,259,317,18,19,20,219,220,221,222,323,208,209]),
torch.tensor([315,314,313,312,311,289,290,294,295,296,297,298,282,299,316,254,257,258,259,317,18,19,20,219,220,221,222,323,208,209]),
torch.tensor([288,289,290,294,295,296,297,298,282,299,316,254,257,258,259,317,18,19,20,219,220,221,222,323,208,209]),
torch.tensor([288,289,290,294,295,296,297,298,282,299,316,254,257,258,259,317,18,19,20,219,220,221,222,323,208,209]),
torch.tensor([288,289,290,294,295,296,297,298,282,299,281,251,250,249,280,300,279,278,277,273,274,240,275,239,106,107,108,109,110,111,112,113,141,142,143,144,145,126,259,317,18,127,19,20,219,128,129,130,137])
]

NORTHERN_ROUTES =[
torch.tensor([128,5,75,6,20,21,2]),
torch.tensor([25,26,0,4,5,6,7]),
torch.tensor([7,77,8,120,9,79,52,332,333,334,59]),
torch.tensor([128,5,75,6,20,129,130]),
torch.tensor([128,5,75,6,20,21,2]),
torch.tensor([3,4,5,75,6,76,7]),
torch.tensor([128,5,75,6,20,21,2]),
torch.tensor([128,5,75,6,20,21,2]),
torch.tensor([7,6,20,21,0,1,2]),
torch.tensor([126,127,197,107,118,70,71,72,73]),
torch.tensor([276,277,278,272,273,274,275,276]),
torch.tensor([128,5,75,6,20,21,2]),
torch.tensor([128,5,75,6,20,21,2]),
torch.tensor([113,119,35,36,37,38,114,39,40,41,42,131,132,133,135,136,137,150,149,148,151,138,139,140,141,142,143,144,145,2]),
torch.tensor([128,5,75,6,20,21,2]),
torch.tensor([128,5,75,6,20,21,2]),
torch.tensor([3,4,5,6,20,21,2]),
torch.tensor([234,105,68,213,214,280,229,230,69,70,71,72,73]),
torch.tensor([128,5,75,6,20,21,2]),
torch.tensor([128,5,75,6,20,21,2]),
torch.tensor([128,5,75,6,20,21,2]),
torch.tensor([3,4,5,6,20,21,2]),
torch.tensor([128,5,75,6,20,21,2]),
torch.tensor([288,338,308,261,262,263,211,178,266,267,287,288]),
torch.tensor([255,303,304,302,246,247,248,249,254,253]),
torch.tensor([3,4,5,6,20,129,130]),
torch.tensor([128,5,75,6,20,21,2]),
torch.tensor([128,5,75,6,20,21,2]),
torch.tensor([128,5,75,6,20,129,130]),
torch.tensor([128,5,75,6,20,129,130]),
torch.tensor([165,270,271,272,273,274,275,276]),
torch.tensor([3,4,5,75,6,76,7]),
torch.tensor([128,5,75,6,20,21,2]),
torch.tensor([231,127,106,107,118,70,71,72,73]),
torch.tensor([128,5,75,6,20,21,2]),
torch.tensor([3,4,5,75,6,20,21,2]),
torch.tensor([128,5,75,6,20,21,2]),
torch.tensor([3,4,5,75,6,76,7]),
torch.tensor([288,289,290,291,292,293,294,50,295,49,237,238,239,255]),
torch.tensor([3,4,5,6,20,21,2]),
torch.tensor([128,5,75,6,20,21,2]),
torch.tensor([2,22,152,153,110,111,112,41,115,116,117,109,155,0,25]),
torch.tensor([128,5,75,6,20,21,2]),
torch.tensor([233,208,209,210,173,235,236,237,238,239,240,241,242,243]),
torch.tensor([233,208,209,210,173,235,236,237,238,239,240,255]),
torch.tensor([233,208,209,210,173,235,236,237,238,239,240,255]),
torch.tensor([233,208,209,210,173,235,236,237,238,239,255]),
torch.tensor([233,208,209,210,173,235,236,237,238,239,240,255]),
torch.tensor([233,208,209,210,173,235,236,237,238,239,255]),
torch.tensor([233,208,209,210,173,235,236,237,238,239,255]),
torch.tensor([233,208,209,210,173,235,236,237,238,239,255]),
torch.tensor([233,208,209,172,173,235,236,237,238,239,240,255]),
torch.tensor([233,208,209,210,173,235,236,237,238,239,255]),
torch.tensor([233,208,209,172,173,235,236,237,238,239,240,255]),
torch.tensor([165,270,271,272,273,274,275,276]),
torch.tensor([276,277,278,272,279,270,165]),
torch.tensor([276,277,278,272,273,274,275,276]),
torch.tensor([288,289,290,291,292,293,294,51,321,322,307]),
torch.tensor([265,60,61,62,63,64,65,66,9,7]),
torch.tensor([276,277,278,272,273,274,275,276])
]

In [4]:
# ANDRE - Change Here
AIRPORT_ROUTES = [
    torch.tensor([9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]),
    torch.tensor([0,1,2,3,4,129,130,131,132,133,134,135,136,137,138,139,140,141]),
    torch.tensor([79,78,84,76,75,74,85,72,71,70,69,68,86,67,66,65,87,88,63,62,89,58,57,55,90,91,92,93,94,95,50,49,48,47,96,97,98,44,43,99,100,101,102,41,103,39,38,37,35,34,33,29]),
    torch.tensor([164,169,168,170,171,172,173,174,175,176,177,178,179,180,181,183,182,185,184,214,215,216,217,218,219,194,195,196,197,198,186,187,188,189,190,209,210,211,212,213,220,203,208,204,205,206,207,202,201,221,222,223,200,199,29]),
    torch.tensor([31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79]),
    torch.tensor([109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,26,5,6,7,27,4,30,107,106,128,105,104,1,29]),
    torch.tensor([24,22,21,20,19,18,17,16,15,14,13,12,11,25,26,5,6,7,8,9]),
    torch.tensor([0,1,2,104,105,106,107,30,4,108,5,6,7,8,9]),
    torch.tensor([164,165,139,166,137,136,135,167,133,132,131,130,129,4,3,28,2,29]),
    torch.tensor([155,156,157,158,159,160,161,162,146,147,148,149,150,151,152,153,154]),
    torch.tensor([0,199,200,223,222,221,201,202,207,206,205,204,208,203,220,213,212,211,210,209,191,192,193,219,218,217,216,224,214,184,185,182,183,181,180,179,178,177,176,175,173,172,171,168,141])
]

MATER_DEI_NORTH_ROUTES = [
    torch.tensor([307,144,90,91,92,93,147,3,4,5]),
    torch.tensor([231,260,259,258,257,256,267,254,253,252,251,249,250,75,111,10,322,98,97,160,161,164,165,166,167,168,169,170,302,83,317,316,318,81,319,2,303,76,3,4,5]),
    torch.tensor([50,49,55,47,46,45,56,43,110]),
    torch.tensor([50,49,55,47,46,45,56,43,110]),
    torch.tensor([134,135,142,141,140,143,139,138,137,112,113,122,121,103,104,105,106,107,108,109,283,284,285,286,287,288,289,290,51]),
    torch.tensor([50,49,55,47,46,45,56,43,110]),
    torch.tensor([89,304,305,78,148,149,152,153,154,155,156,157,158,76,3,4,5]),
    torch.tensor([279,314,9,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,217,299,201,200,199,198,197,196,195]),
    torch.tensor([134,135,142,141,140,143,139,138,137,112,113,122,114,115,116,117]),
    torch.tensor([230,261,262,263,264,265,266,37,38,39,40,41,42,43,44,45,46,47,48,49,50]),
    torch.tensor([279,280,281,275,274,273,272,271,270,269,282,248,240,234,60,30,29,27,61,62,63,64,65,22,21,20,19,66,67,68,16,15,69,70,71,72,13,73,11]),
    torch.tensor([279,314,9,0,1,84,83,315,316,318,81,2,303,162,76,3,4,5]),
    torch.tensor([7,294,160,161,164,165,166,167,168,169,170,171,172,0,9,314,280,281,275,274,273,272,271,270,269,282,248,240,229,235,236,228,218,219,220,221,222,223,224,225,226,227,77,74,100,10,101,119,120,121,122,114,115,118,123,136,124,125,126,127,130,131,132,133,134]),
    torch.tensor([50,49,55,47,46,45,56,43,110]),
    torch.tensor([195,215,199,200,201,299,189,188,187,190,185,184,182,181,191,192,179,178,177,193,194,175,174,173,9,314,280,281,275,274,321,308,309,310,272,271,270,269,282,33,233,232,231]),
    torch.tensor([50,49,55,47,46,45,56,43,110]),
    torch.tensor([12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,248,240,234,268,269,270,271,272,273,274,275,311,312,313,276,277,278]),
    torch.tensor([7,294,160,320,98,99,100,163,204,303,76,3,4,5]),
    torch.tensor([6,145,158,157,156,154,159,153,152,150,79,85,80,96,97,160,161,2,81,82,83,84,8,1,0,173,174,175,176,177,178,179,180,181,182,183,184,185,205,206,207,208,209,210,211,212,213,214,189,299,201,200,199,198,197,196,195]),
    torch.tensor([7,145,146,94,93,92,91,95,307]),
    torch.tensor([50,49,55,47,46,45,56,43,42,41,40,39,57,38,37,36,58,59,35,34,268,269,282,248,240,229,235,236,228,100,98,96,80,85,86,148,149,152,150,151,87,306,88]),
    torch.tensor([291,289,292,287,293,285,284,283,109,108,107,106,105,104,103,102,111,10,75,237,238,239,251,252,253,254,255,256,257,258,259,260,230]),
    torch.tensor([89,304,305,78,148,149,152,150,151,87,306,88]),
    torch.tensor([195,215,199,200,201,299,189,214,213,212,211,210,300,209,208,207,206,301,185,184,182,181,191,192,179,178,177,193,194,175,174,173,9,314,280,281,275,276,277,278])
]


MATER_DEI_SOUTH_ROUTES = [
    torch.tensor([7,294,160,320,319,2,81,82,83,84,8,1,0,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,299,201,200,199,198,197,196,195]),
    torch.tensor([230,261,262,263,264,265,266,37,38,39,40,41,42,43,44,45,46,47,48,49,50]),
    torch.tensor([50,49,55,47,46,45,56,43,110]),
    torch.tensor([12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,248,240,229,235,236,228,100,10,322,98,96,80,85,86,148,149,152,153,154,155,156,157,158,76,3,4,5]),
    torch.tensor([7,294,160,161,164,165,166,167,168,169,170,171,172,0,9,314,280,281,275,276,277,278]),
    torch.tensor([50,49,55,47,46,45,56,43,110]),
    torch.tensor([50,49,55,47,46,45,56,43,110]),
    torch.tensor([7,145,146,94,93,92,91,95,307]),
    torch.tensor([231,260,259,258,257,256,267,254,253,252,251,249,250,75,111,10,163,204,303,162,76,3,4,5]),
    torch.tensor([50,49,55,47,46,45,56,43,110]),
    torch.tensor([50,49,55,47,46,45,56,43,42,41,40,39,57,38,37,36,58,59,35,34,268,269,270,271,272,273,274,275,276,277,278]),
    torch.tensor([134,135,142,141,140,143,139,138,137,112,113,122,121,103,104,105,106,107,108,109,283,284,285,286,287,288,289,290,51]),
    torch.tensor([279,314,9,0,1,84,83,317,316,318,81,2,303,76,3,4,5]),
    torch.tensor([195,215,199,200,201,299,189,188,187,190,185,184,182,181,191,192,179,178,177,193,194,175,174,173,0,172,170,302,83,315,316,318,81,2,303,76,3,4,5]),
    torch.tensor([50,49,55,47,46,45,56,43,110]),
    torch.tensor([6,145,158,157,156,154,159,153,152,150,151,87,306,88]),
    torch.tensor([89,304,305,78,148,149,152,150,79,85,80,96,97,98,99,100,10,101,119,120,121,122,114,115,118,123,136,124,125,126,127,130,131,132,133,134]),
    torch.tensor([291,289,292,287,293,285,284,283,109,108,107,106,105,104,103,102,111,10,75,237,238,239,251,252,253,254,255,256,257,258,259,260,230]),
    torch.tensor([134,135,142,141,140,143,139,138,137,112,113,122,121,103,102,111,10,322,98,99,100,218,219,220,221,222,223,224,225,226,227,77,74,228,229,240,234,60,30,29,27,61,62,63,64,65,22,21,20,19,66,67,68,16,15,69,70,71,72,13,73,11]),
    torch.tensor([134,135,142,141,140,143,139,138,137,112,113,122,114,115,116,117]),
    torch.tensor([279,280,281,311,312,313,276,277,278]),
    torch.tensor([307,144,90,91,92,93,147,3,4,5]),
    torch.tensor([50,49,55,47,46,45,56,43,110]),
    torch.tensor([195,215,199,200,201,299,217,189,214,213,212,211,210,300,209,208,207,206,301,185,184,182,181,191,192,179,178,177,193,194,175,174,173,9,314,280,281,275,274,321,308,309,310,272,271,270,269,282,33,233,232,231])
]

GOZO_ROUTES = [
    torch.tensor([216,217,218,219,220,221,122,222,223,124,123,126,224,40]),
    torch.tensor([47,1,2,3,116,117,118,119,120,121,122,221,220,219,218,217,216]),
    torch.tensor([23,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,23]),
    torch.tensor([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]),
    torch.tensor([187,186,185,188,183,182,189,190,144,145,146,147,148,149,150,151,143,142,152,203]),
    torch.tensor([23,101,245,246,112,111,110,109,108,107]),
    torch.tensor([23,24,25,26,27,28,29,6,30,31,32,33,34,35,36,37,38]),
    torch.tensor([107,108,109,110,111,112,113,114,115,23]),
    torch.tensor([38,37,36,35,48,225,226,52,227,228,65,66,15]),
    torch.tensor([23,101,245,246,112,111,110,109,108,107]),
    torch.tensor([15,14,13,72,73,242,243,244,11,10,16,238,9,17,8,7,6,5,18,19,20,21]),
    torch.tensor([23,101,245,246,112,111,110,109,108,107]),
    torch.tensor([15,66,65,74,63,62,61,75,77,78,79,81,80,59,76,58,53,226,225,48,35,36,37,38]),
    torch.tensor([23,101,245,246,112,111,110,109,108,107]),
    torch.tensor([23,101,245,246,112,111,110,109,108,107]),
    torch.tensor([23,101,245,246,112,111,110,109,108,107]),
    torch.tensor([38,37,36,35,48,49,50,51,52,53,54,58,59,60,61,62,63,64,65,66,15]),
    torch.tensor([23,24,25,26,230,231,232,233,234,235,236,237,238,9,239,240,241,70,71,13,72,73,69,68,67,66,15]),
    torch.tensor([23,101,100,98,102,96,103,104,105,106,107]),
    torch.tensor([137,138,139,140,141,142,143,144,179,180,181,193,194,195,196,197,198,199,200,201,202,183,184,185,186,187]),
    torch.tensor([191,1,2,3,4,5,6,30,31,32,41,42,28,27,26,43,44,45,46,23]),
    torch.tensor([22,1,136,192,141,204,152,140,139,138,137]),
    torch.tensor([40,229,126,127,124,125,123,122,128,121,134,135,119,118,117,116,20,21]),
    torch.tensor([23,101,245,246,112,111,110,109,108,107]),
    torch.tensor([23,101,245,246,112,111,110,109,108,107]),
    torch.tensor([23,101,245,246,112,111,110,109,108,107])
]

In [5]:
class Config(dict):
    def __init__(self, *args, **kwargs):
        super(Config, self).__init__(*args, **kwargs)
        for key, value in self.items():
            if isinstance(value, dict):
                self[key] = Config(value)
            elif isinstance(value, list):
                # Convert dict elements in lists to Config objects
                self[key] = [Config(item) if isinstance(item, dict) else item for item in value]

    __getattr__ = dict.get

cfg = Config({
    'ppo': {
        'n_iterations': 200,
        'val_period': 10,
        'n_epochs': 1,
        'minibatch_size': 2,
        'horizon': 26,
        'epsilon': 0.2,
        'use_gae': False,
        'gae_lambda': 0.95
    },
    'discount_rate': 0.95,
    'diff_reward': False,
    'baseline_lr': 0.0005,
    'entropy_weight': 0,
    'batch_size': 2,
    'reward_scale': 1.0,
    'lr': 0.0016134816080499328,
    'decay': 0.0008404361781997002,
    'optimizer': 'Adam',
    'eval': {
        # ANDRE - CHANGE HERE
        'n_routes': 60,
        'min_route_len': 7,
        'max_route_len': 71
    },
    'dataset': {
        'type': 'mumford',
        'kwargs': {
            'path': 'datasets/mumford_dataset/Instances',
            'city': 'Gozo'
        }
    },
    'experiment': {
        'logdir': 'training_logs',
        'anomaly': False,
        'cpu': False,
        'seed': 0,
        'symmetric_routes': False
    },
    'defaults': [
        '_self_',
        {
            'cost_function': {
                'type': 'mine',
                'kwargs': {
                    'mean_stop_time_s': 0,
                    'avg_transfer_wait_time_s': 300,
                    'demand_time_weight': 0.5,
                    'route_time_weight': 0.5,
                    'constraint_violation_weight': 5.0,
                    'variable_weights': False,
                    'pp_fraction': 0.33,
                    'op_fraction': 0.33
                }
            }
        }
    ]
})


In [6]:
class Config(dict):
    def __init__(self, *args, **kwargs):
        super(Config, self).__init__(*args, **kwargs)
        for key, value in self.items():
            if isinstance(value, dict):
                self[key] = Config(value)
    __getattr__ = dict.get

# Minimal experiment configuration
exp_dc = Config({
    'cost_function': {
        'type': 'mine',  # Specify the cost function type as 'mine'
        'kwargs': {}     # Any additional parameters can be added here
    },
    'symmetric_routes': False,    # or False, based on your requirement
    'low_memory_mode': False
})


In [7]:
class Config(dict):
    __getattr__ = dict.get

# ANDRE - CHANGE HERE
Dataset_Info = Config({
    'csv': True,
    'n_routes': 60,
    'min_route_len': 7,
    'max_route_len': 71,
    'type': 'mumford',
    'path': 'C:/Users/Owner/ICT5012 - Disseration/transit_learning-master/CEC2013Supp/Instances/Morning_Model',
    'city': 'northern_routes'
})

In [8]:
exp_cfg = cfg.experiment

In [9]:
if exp_cfg.get('cpu', False) or not torch.cuda.is_available():
    device = torch.device("cpu")
else:
    device = torch.device("cuda")

In [10]:
if 'model' in cfg:
    # setup the model
    model = build_model_from_cfg(cfg['model'], exp_cfg)
    if 'weights' in cfg.model:
        model.load_state_dict(torch.load(cfg.model.weights,map_location=device))
    elif weights_required and cfg.model.route_generator.type != 'RandomPathCombiningRouteGenerator': raise ValueError("model weights are required but not provided")
else:
    model = None

In [11]:
# setup the cost function
low_mem_mode = exp_cfg.get('low_memory_mode', False)
cost_obj = get_cost_module_from_cfg(exp_dc.cost_function, low_mem_mode,
                                    exp_cfg.symmetric_routes)

low_memory_mode: False


In [12]:
cost_obj.to(device)
if model is not None:
    model.to(device)

In [13]:
test_ds = get_dataset_from_config(Dataset_Info)
test_dl = DataLoader(test_ds, batch_size=10)


In [14]:
n_samples = cfg.get('n_samples', None)

In [15]:
sbs = cfg.get('sample_batch_size', cfg.batch_size)

In [16]:
for data in tqdm(test_dl):
    if device is not None and device.type != 'cpu':
        data = data.cuda()
    start_time = time.time()
    state = RouteGenBatchState(data, cost_obj, cfg.eval.n_routes, 
                                   cfg.eval.min_route_len, 
                                   cfg.eval.max_route_len)

100%|██████████| 1/1 [00:08<00:00,  8.50s/it]


In [17]:
import torch

# ANDRE - CHANGE HERE
# Your list of route tensors (on CUDA). Each tensor is 1D.
routes_list = NORTHERN_ROUTES

# Determine maximum route length among all routes.
max_length = max(route.numel() for route in routes_list)
print("Max route length:", max_length)

# Pad each route with -1 so that all routes have the same length.
padded_routes = []
for route in routes_list:
    pad_length = max_length - route.numel()
    if pad_length > 0:
        padded_route = torch.cat([route, -1 * torch.ones(pad_length, dtype=route.dtype, device=route.device)])
    else:
        padded_route = route
    padded_routes.append(padded_route)

# Stack the padded routes to form a tensor of shape [n_routes, max_length].
routes_tensor = torch.stack(padded_routes)

# Add a batch dimension so the final tensor has shape [1, n_routes, max_length].
final_routes_tensor_correct = routes_tensor.unsqueeze(0)


Max route length: 30


In [18]:
state.add_new_routes(final_routes_tensor_correct)


In [19]:
MEAN_STOP_TIME_S = 0
AVG_TRANSFER_WAIT_TIME_S = 300
UNSAT_PENALTY_EXTRA_S = 3000
EPSILON = 1e-6


# Start by Obtaining Passenger Cost

# Load Demand Matrix from State
demand_matrix = state.demand
# Load Drive Times Matrix from State
drive_times_matrix = state.drive_times

# Calculate amount of demand at each number of transfers
trips_at_transfers = torch.zeros(state.batch_size, 4, device=device)
# Load Number of transfers
n_transfers = state.n_transfers.clone()
# Label instances where no path exists as nopath
nopath = ~state.has_path
# Obtain number of unsatisfied demand
n_transfers[nopath] = 3
for ii in range(3):
    d_i = (demand_matrix * (n_transfers == ii)).sum(dim=(1,2))
    trips_at_transfers[:, ii] = d_i
    
d_un = (demand_matrix * (n_transfers > 2)).sum(dim=(1, 2))
trips_at_transfers[:, 3] = d_un


# Load transit_times (equivalent to shortest distances based on floyd_warshall)
trip_times = state.transit_times
# Adjust 'inf' trip_time values to 0 if nopath exists
trip_times[nopath] = 0
# Caclulate demand_time matrix
demand_time = demand_matrix * trip_times
# Calculate total_demand_time
total_dmd_time = demand_time.sum(dim=(1, 2))
# Calculate number of total_transfers
demand_transfers = demand_matrix * state.n_transfers
total_transfers = demand_transfers.sum(dim=(1, 2))
# Calculate unserved demand
unserved_demand = (demand_matrix * nopath).sum(dim=(1, 2))
# Calculate total demand
total_demand = demand_matrix.sum(dim=(1,2))
# Calculate served demand
served_demand = (state.has_path * demand_matrix).sum(dim = (1,2))
# Calculate Mean Demand Time
mean_demand_time = total_dmd_time/ (served_demand + EPSILON)
# Calculate time_normalizer
time_normalizer = state.drive_times.flatten(1,2).max(1).values
# Calculate Demand Cost
demand_cost = (mean_demand_time*served_demand) + (unserved_demand*time_normalizer*2)
# Calculate Demand Cost (Before Normalizer)
demand_cost /= total_demand
# Calculate Demand Cost (After Normalizer)
demand_cost = demand_cost/time_normalizer

# Start by Obtaining Operator Cost
n_routes = state.n_routes_to_plan
# Calculate Route Cost (Before Normalizer)
route_cost = state.total_route_time
# Calculate Route Cost (After Normalizer)
route_cost = route_cost/(time_normalizer * n_routes + 1e-6)

# Passenger and Operator Cost
demand_weight = 0.5
route_weight = 0.5
Passenger_y_Operator_Cost = (demand_weight*demand_cost) + (route_weight*route_cost)


# Calculate Constraint Violation

# Number of Uncovered Demand Nodes
frac_uncovered = state.get_n_disconnected_demand_edges() / state.n_demand_edges
# Calculate Constraint Violation
const_viol_cost = frac_uncovered + 0.1 * (frac_uncovered > 0)

# Calculate final cost
cost = Passenger_y_Operator_Cost + (const_viol_cost * 5)

In [20]:
state.get_n_disconnected_demand_edges()

tensor([10821])

In [21]:
state.n_demand_edges

tensor([11936])

In [22]:
d_un

tensor([12225.])

In [23]:
total_demand

tensor([13571.])

In [24]:
cost = cost
C_p = mean_demand_time/60
C_o = state.total_route_time/60
d_0 = (trips_at_transfers[:, 0]/total_demand)*100
d_1 = (trips_at_transfers[:, 1]/total_demand)*100
d_2 = (trips_at_transfers[:, 2]/total_demand)*100
d_unsat = (d_un/total_demand)*100
disconnected_node_pairs = state.get_n_disconnected_demand_edges()
stops_out_of_bounds = min(state.max_route_len - max_length,0)

In [25]:
# Create a dictionary with your desired column headers.
data = {
    "Cost": cost[0].item(),
    "C_p": C_p[0].item(),
    "C_o": C_o[0].item(),
    "0 Tr. (%)": d_0[0].item(),
    "1 Tr. (%)": d_1[0].item(),
    "2 Tr. (%)": d_2[0].item(),
    "Unsat. Demand (%)": d_unsat[0].item(),
    "Disc. NP": disconnected_node_pairs[0].item(),
    "Max RL": stops_out_of_bounds != 0
}

# Create a pandas DataFrame.
df = pd.DataFrame([data])  # each key becomes a column header

# Display the DataFrame.
print(df)

       Cost        C_p          C_o  0 Tr. (%)  1 Tr. (%)  2 Tr. (%)  \
0  6.051338  26.993658  1239.366699   4.546459   3.573797   1.797952   

   Unsat. Demand (%)  Disc. NP  Max RL  
0          90.081795     10821   False  


In [26]:
from tabulate import tabulate 
print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))

+---------+---------+---------+-------------+-------------+-------------+---------------------+------------+----------+
|    Cost |     C_p |     C_o |   0 Tr. (%) |   1 Tr. (%) |   2 Tr. (%) |   Unsat. Demand (%) |   Disc. NP | Max RL   |
|---------+---------+---------+-------------+-------------+-------------+---------------------+------------+----------|
| 6.05134 | 26.9937 | 1239.37 |     4.54646 |      3.5738 |     1.79795 |             90.0818 |      10821 | False    |
+---------+---------+---------+-------------+-------------+-------------+---------------------+------------+----------+
